# Introduccion

Este notebook contiene los procesos que se han realizado para limpiar las bases de datos de contratos del sistema RENOVA de la SUPERINTENDENCIA DE VIGILANCIA Y SEGURIDAD PRIVADA DE COLOMBIA. Contiene las funciones e scripts para cargar los datasets en Jupyter o Colab, a partir de archivos en formato CSV. 

Los datos del RUES provienen del sistema de Datos Abiertos:

(https://www.datos.gov.co/browse?Informaci%C3%B3n-de-la-Entidad_Nombre-de-la-Entidad=Superintendencia+de+Vigilancia+y+Seguridad+Privada&sortBy=newest)

Los datos son los correspondientes a Empresas registradas como Servicios de vigilancia y Seguridad Privada y las empresas en el Registro como proveedores de equipos para el sector de Vigilancia y Seguridad privada.

Esta es la segunda etapa del trabajo con estos datos. 

El objetivo de éste notebook es:
  - Identificar cuales empresas están registradas en ambos sistemas y cuales no.

Los datos de salida del notebook son:
  - Una tabla con los datos de RENOVA limpios, listos para ser visualizados en el Buscador de Servicios. Se debe identificar cuales empresas tienen registro en el RUES

In [2]:
# First import the required libraries.
import pandas as pd
import numpy as np

# To load an manage files:
import io
import os

In [3]:
# Ajustar No Maximo de Columnas para mostrar
# pd.get_option("display.max_columns")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_columns', 20) # reset to default

In [3]:
# Ajustar el tamaño máximo de columna que se muestra
#pd.get_option('display.max_colwidth')
#pd.set_option('display.max_colwidth', -1) 
#pd.set_option('display.max_colwidth', 50)  # reset to default

In [4]:
pd.get_option('display.max_rows')

60

In [5]:
# Ajustar el número máximo de filas para mostrar
#pd.get_option('display.max_rows')
#pd.set_option('display.max_rows', None) # mostrar todas
pd.set_option('display.max_rows', 60) # reset to default

In [ ]:
#para poder ver los montos en valores con todas las cifras significativas en vez de notación científica
pd.set_option('display.float_format',  '{:,.2f}'.format)

# Step 1: **Cargar Archivos**

## Funciones Complementarias

### Funcion para Ajustar NITS a 14 caracteres

In [4]:
# using strings of 14 characters lenght in order to match what we will get from the RUES database:

def extender_nit_string(nit):
    nit = str(nit)
    return (14-len(nit))*'0' + nit

# driver code
nit_list = ['1', '2', '3']
  
# Map method returns a map object
# so we cast it into list using list()
output_list = list(map(extender_nit_string, nit_list))
print(output_list)

['00000000000001', '00000000000002', '00000000000003']


### Funcion para calcular los dígitos de verificacion

In [5]:
# Funcion para calcular el dígito de verificación para el Registro Trubutario (RUT) o el Numero de Identificacion Tributaria (NIT)
# El valor de entrada es un número entre 5 y 15 cifras
def calcular_digito_ver(num):

    BASE = [3, 7, 13, 17, 19, 23, 29, 37, 41, 43, 47, 53, 59, 67, 71]
    DIVIDER = 11

    index = 0
    accumulator = 0
    num = str(num)
    num = [x for x in num]
    num = num[::-1]
    #print (type(num), num)
    for char in num:
        calculation = int(char) * BASE[index]
        accumulator += calculation
        index += 1

    remainder = accumulator % DIVIDER

    if (remainder>1):
        return (11-remainder)
    else:
        return str(remainder)

In [6]:
calcular_digito_ver(800233036)

7

### Funcion para Ajustar Formato de CIIU Codes

In [7]:
def limpiar_ciuu_code(text):
    text = str(text)
    if text == 'nan':
        return np.nan
    elif len(text) == 1:
        return '0' + text + '00'
    elif len(text) == 2:
        return text + '00'
    elif len(text) < 4:
        return (4-len(text))*'0' + text
    elif len(text) > 4:
        return text[0:4]
    else:
        return text

limpiar_ciuu_code('01')
        

'0100'

### Diccionario de Tipos de Identificacion

In [10]:
id_dict = {'00': 'NO DEFINIDO',
 '01': 'CEDULA DE CIUDADANIA',
 '02': 'NIT',
 '06': 'SIN IDENTIFICACION',
 '03': 'CEDULA DE EXTRANJERIA',
 '05': 'PASAPORTE',
 '04': 'TARJETA DE IDENTIDAD',
 '09': 'PERMISO ESPECIAL DE PERMANENCIA',
 '10': 'PERMISO POR PROTECCION TEMPORAL',
 '08': 'DOCUMENTO EXTRANJERO',
 '07': 'REGISTRO CIVIL DE NACIMIENTO'}
id_dict

{'00': 'NO DEFINIDO',
 '01': 'CEDULA DE CIUDADANIA',
 '02': 'NIT',
 '06': 'SIN IDENTIFICACION',
 '03': 'CEDULA DE EXTRANJERIA',
 '05': 'PASAPORTE',
 '04': 'TARJETA DE IDENTIDAD',
 '09': 'PERMISO ESPECIAL DE PERMANENCIA',
 '10': 'PERMISO POR PROTECCION TEMPORAL',
 '08': 'DOCUMENTO EXTRANJERO',
 '07': 'REGISTRO CIVIL DE NACIMIENTO'}

## Load CSV into colab 
From Google Drive only (DO NOT RUN this section if in Jupyter):

In [ ]:
# For Using Google Drive, (Only if executing notebook from Google colab):
from google.colab import drive
drive.mount('/content/drive')

# after that:
# <--- Refresh mounted Drive
# <--- Look for file and get the path link

Mounted at /content/drive


In [ ]:
# Get the current directory
print(os.getcwd())
# Check if the directory exists
print(os.path.exists('../'))

/content
True


In [ ]:
ls

In [ ]:
cd /content/drive/MyDrive/Observatorio SPVS/

## Load files from OneDrive

In [8]:
pwd

'd:\\DS4A\\SuperVigilancia\\OneDrive - Superintendencia de Vigilancia\\Proyecto Observatorio 2022\\5.Notebooks\\1_Servicios_Vigilados'

In [9]:
cd "../../3.Bases_de_datos_Internas/RENOVA-Datos_Abiertos-Empresas_de_Vigilancia/"

d:\DS4A\SuperVigilancia\OneDrive - Superintendencia de Vigilancia\Proyecto Observatorio 2022\3.Bases_de_datos_Internas\RENOVA-Datos_Abiertos-Empresas_de_Vigilancia


In [4]:
ls

 Volume in drive D has no label.
 Volume Serial Number is 5E36-13E0

 Directory of d:\DS4A\SuperVigilancia\OneDrive - Superintendencia de Vigilancia\Proyecto Observatorio 2022\3.Bases_de_datos_Internas\RENOVA-Datos_Abiertos-Empresas_de_Vigilancia

10/06/2022  09:50 AM    <DIR>          .
10/06/2022  09:50 AM    <DIR>          ..
10/10/2022  12:56 PM    <DIR>          0.Raw_dataset
10/15/2022  01:10 AM    <DIR>          1.Clean_dataset
09/23/2022  10:00 AM    <DIR>          2.Processed_dataset
10/06/2022  09:42 AM    <DIR>          Datos Caso Servicios
05/02/2022  01:38 PM           490,300 Datos Empresas Servicios.zip
03/17/2022  02:28 PM          (28,434) READ ME FIRST- CUIDADO !.docx
               2 File(s)        518,734 bytes
               6 Dir(s)  389,077,819,392 bytes free


In [11]:
with open("./1.Clean_dataset/Empresas_de_Vigilancia_con_y_Sin_Armas__Servicios_Comunitarios__Esc.csv", "r", encoding='utf-8') as file:
    #read_data = file.readlines()
    #for line in read_data:
        
    Servicios = pd.read_csv(file, encoding='utf8', dtype=str, sep =",", parse_dates=True, infer_datetime_format=True)
#close the file (not Necesary)
#f.close()
print(Servicios.shape)
Servicios.head(3)

(1358, 12)


,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL
0,8600654643,360 GRADOS SEGURIDAD LTDA,NO REGISTRA,EMPRESA DE VIGILANCIA Y SEGURIDAD PRIVADA,RENOVACIÓN,P,CALLE 134 BIS No. 19 - 51 BARRIO CONTADOR,BOGOTÁ D.C.,BOGOTÁ D.C.,5190396,g.general@360seguridad.com,CARLOS ALBERTO DELGADILLO GONZALEZ
1,9010567354,361 SEGURIDAD LTDA,NO REGISTRA,EMPRESA DE VIGILANCIA Y SEGURIDAD PRIVADA,LICENCIA,P,PARQUE INDUSTRIAL TERRAPUERTO LOCAL 6 PARQUE L...,CUNDINAMARCA,COTA,3212701079 3176480812 3213136712,jm_algarra@hotmail.com,JOSE MAURICIO ALGARRA GALEANO
2,8603518121,7-24 LTDA,NO REGISTRA,EMPRESA DE VIGILANCIA Y SEGURIDAD PRIVADA,RENOVACIÓN,P,CALLE 108 No. 8 - 45,BOGOTÁ D.C.,BOGOTÁ D.C.,7447724 5203588,"gerente@siete24.com - gerencia@siete24,com",JULIO ALEJANDRO MOLANO KISHEL


In [172]:
with open("./1.Clean_dataset/Empresas_Inscritas_en_el_Registro_como_Fabricaci_n__Comercializaci_.csv", "r", encoding='utf-8') as file:
    #read_data = file.readlines()
    #for line in read_data:
        
    Proveedores = pd.read_csv(file, encoding='utf8', dtype=str, sep =",", parse_dates=True, infer_datetime_format=True)
#close the file (not Necesary)
#f.close()
print(Proveedores.shape)
Proveedores.head(3)

(1207, 12)


,NIT,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,SECCIONES,TIPO OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRONICO - RUES,REPRESENTANTE LEGAL O ADMINISTRADOR
0,805016312,SEGURIDAD J.J. LTDA - EN LIQUIDACION,NO REGISTRA,INSCRITA EN EL REGISTRO,IMPORTACION FABRICACION COMERCIALIZACION INSTA...,P,CL. 28N # 6 BN-54,VALLE DEL CAUCA,CALI,O,NO REGISTRA,JOSE JAVIER SEPULVEDA MURILLO
1,9009663510,WIEX TECHNOLOGIES S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,IMPORTACIÓN,P,CARRERA 57 No. 4B -16,BOGOTÁ D.C.,BOGOTÁ D.C.,3464656,wiextech@gmail.com,WILLIAM HERIBERTO REYES MORENO
2,9008462722,SECURICOL SERVICES S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,IMPORTACIÓN COMERCIALIZACIÓN E INSTALACIÓN,P,CARRERA 5 No. 104 -39,BOGOTÁ D.C.,BOGOTÁ D.C.,7446240,mauricio.hernandez@securicol.com,VICTOR MAURICIO HERNÁNDEZ PRIETO


In [10]:
cd "../../4.Bases_de_datos_Externas/Camaras_de_Comercio-Registro_Unico_de_Empresas_RUES/"

d:\DS4A\SuperVigilancia\OneDrive - Superintendencia de Vigilancia\Proyecto Observatorio 2022\4.Bases_de_datos_Externas\Camaras_de_Comercio-Registro_Unico_de_Empresas_RUES


In [408]:
with open("./1.Clean_dataset/empresas_registradas_RUES_clean_ciiu.csv", "r", encoding='utf-8') as file:
    #read_data = file.readlines()
    #for line in read_data:
        
    Registro_RUES = pd.read_csv(file, encoding='utf8', dtype=str, sep =",", parse_dates=True, infer_datetime_format=True)
#close the file (not Necesary)
#f.close()
print(Registro_RUES.shape)
Registro_RUES.head(3)

(4783254, 25)


,Unnamed: 0,codigo_camara,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,razon_social,codigo_estado_matricula,estado_matricula,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,ciiu3,ciiu4,fecha_cancelacion,codigo_clase_identificacion,clase_identificacion,numero_identificacion,nit_propietario,digito_verificacion,codigo_camara_propietario,camara_comercio_propitario,matrícula_propietario,Codigo_tipo_propietario,tipo_propietario,categoria_matricula
0,0,23,HUILA,290479,2022,2022-07-22,MANTENIMARMOL,3,CANCELADA,4330,4290,NaN,NaN,20220722.0,01,CEDULA DE CIUDADANIA,1075287105,00001075287105,0.0,23,HUILA,290478,NaN,NaN,ESTABLECIMIENTO DE COMERCIO
1,1,5,BUCARAMANGA,410242633,2022,2022-04-25,TIENDA ANGELITOS CON AMOR,1,ACTIVA,4759,4771,NaN,NaN,NaN,02,NIT,37896777,00000037896777,3.0,5,BUCARAMANGA,177224,1,PROPIETARIO ÚNICO,ESTABLECIMIENTO DE COMERCIO
2,2,45,SUR Y ORIENTE DEL TOLIMA,23329,2022,2022-03-29,BANCO DAVIVIENDA S.A AGENCIA CHAPARRAL,1,ACTIVA,6412,NaN,NaN,NaN,NaN,02,NIT,860034313,00000860034313,7.0,4,BOGOTA,276917,NaN,NaN,AGENCIA


In [409]:
Registro_RUES.drop(columns = ['Unnamed: 0'], inplace=True)

In [410]:
Registro_RUES = Registro_RUES[[ 'clase_identificacion', 'numero_identificacion', 'nit_propietario','razon_social', 
       'cod_ciiu_act_econ_pri', 'cod_ciiu_act_econ_sec', 'ciiu3', 'ciiu4', 
       'camara_comercio', 'matricula', 'ultimo_ano_renovado', 'fecha_renovacion', 'estado_matricula', 'fecha_cancelacion', 'categoria_matricula']]

In [411]:
Registro_RUES = Registro_RUES.sort_values(by=['ultimo_ano_renovado', 'fecha_renovacion'], ascending=False).sort_values(by=['estado_matricula'])
Registro_RUES.head()

,clase_identificacion,numero_identificacion,nit_propietario,razon_social,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,ciiu3,ciiu4,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,fecha_cancelacion,categoria_matricula
4783253,CEDULA DE CIUDADANIA,39622413,00000039622413,CASINO ZEUS FUSA,9200,NaN,NaN,NaN,BOGOTA,3574786,0,--,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
3693571,CEDULA DE CIUDADANIA,65634159,00000065634159,PEPITOS.COM,5619,NaN,NaN,NaN,IBAGUE,330473,2021,2021-05-19,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
3681712,CEDULA DE CIUDADANIA,24228785,00000024228785,NUTRIMASCOTAS KILO A KILO,4759,9200,NaN,NaN,SOGAMOSO,40921,2021,2021-05-19,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
3677352,CEDULA DE CIUDADANIA,70879621,00000070879621,BORDADOS ROYAL,1313,NaN,NaN,NaN,ABURRA SUR,213074,2021,2021-05-19,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
3657753,CEDULA DE CIUDADANIA,53082269,00000053082269,ALQUILER DE LAVADORAS J R R,4761,4759,NaN,NaN,BOGOTA,2078356,2021,2021-05-19,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO


# Step 2: Arreglar Formato y Consistencia de los datos

In [212]:
Servicios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1358 entries, 0 to 1357
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NIT O CC             1358 non-null   object
 1   RAZÓN SOCIAL         1358 non-null   object
 2   ACRÓNIMO             1357 non-null   object
 3   TIPO DE SERVICIO     1358 non-null   object
 4   TIPO DE TRÁMITE      1357 non-null   object
 5   TIPO DE OFICINA      1358 non-null   object
 6   DIRECCIÓN            1358 non-null   object
 7   DEPARTAMENTO         1358 non-null   object
 8   CIUDAD               1358 non-null   object
 9   TELÉFONO             1358 non-null   object
 10  CORREO ELECTRÓNICO   1351 non-null   object
 11  REPRESENTANTE LEGAL  1358 non-null   object
dtypes: object(12)
memory usage: 127.4+ KB


In [213]:
Servicios["NIT O CC"].describe()

count           1358
unique          1312
top       8600405761
freq              23
Name: NIT O CC, dtype: object

In [19]:
Servicios[Servicios["NIT O CC"].isna()]

,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL
941,NaN,GMW SECURITY RENT A CAR LTDA,NO REGISTRA,DEPARTAMENTO DE SEGURIDAD,LICENCIA,P,Carrera 15 No 124-30 OF 407,BOGOTÁ D.C.,BOGOTÁ D.C.,6113372 3101294 3116419125,contabilidad@gmwblindajes.com,LUIS WILSON GONZALEZ


In [214]:
# Se le asigna el valor que está en la misma base de datos. Ojo, es un departamento de seguridad de una empresa blindadora, por eso está repetido
#Servicios.loc[Servicios["NIT O CC"].isna(), "NIT O CC"] = 8301298270
Servicios.loc[941, "NIT O CC"] = 8301298270


In [215]:
Proveedores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1207 entries, 0 to 1206
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NIT O CC             1207 non-null   object
 1   RAZÓN SOCIAL         1207 non-null   object
 2   ACRÓNIMO             1203 non-null   object
 3   TIPO DE SERVICIO     1207 non-null   object
 4   SECCIONES            1197 non-null   object
 5   TIPO DE OFICINA      1207 non-null   object
 6   DIRECCIÓN            1207 non-null   object
 7   DEPARTAMENTO         1207 non-null   object
 8   CIUDAD               1207 non-null   object
 9   TELÉFONO             1207 non-null   object
 10  CORREO ELECTRÓNICO   1207 non-null   object
 11  REPRESENTANTE LEGAL  1206 non-null   object
dtypes: object(12)
memory usage: 113.3+ KB


In [216]:
# Que pasa con los proveedores que no tienen seccion registrada?
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.upper()
Proveedores[Proveedores["SECCIONES"].isna()]

,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,SECCIONES,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL
212,8040147678,FOCUS INGENIERIA LTDA,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CL. 41 # 37-16 BARRIO CABECERA DEL LLANO,SANTANDER,BUCARAMANGA,7013055 7013056,hlopez@focusingenieria.com,HERMES LOPEZ URBINA
453,8301475471,E TRAINING S A S,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 59 A BIS N° 5 - 53 OF 206,BOGOTÁ D.C.,BOGOTÁ D.C.,2565899,gabrielc@etraining.co,DAIRO DE JESUS RODRIGUEZ BONILLA
584,9000706454,PARKING OLE S.A.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CARRERA 48 A No. 10 SUR-161,ANTIOQUIA,MEDELLIN,6046673,"evelandia@olemovilidad.com.co, mardila@olemov...",NELSON ALFONSO GUILLEN PEÑUELA
886,9004053933,MEDINNOVA SAS,NaN,INSCRITA EN EL REGISTRO,NaN,P,CLLE 114 No. 56 - 26 - 402,BOGOTÁ D.C.,BOGOTÁ D.C.,7431451,info@medinnova.co,FERMIN EDUARDO VARGAS TRIANA
916,9004269133,INCOSIS S A S,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 4 SUR No. 71D-64,BOGOTÁ D.C.,BOGOTÁ D.C.,4142186,gmr@incosis.co,GINA MARCELA RODRIGUEZ SARMIENTO
935,9004485619,CAT COLOMBIA SOLUTIONS S.A.S,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 135C No. 9A-81,BOGOTÁ D.C.,BOGOTÁ D.C.,3183061180 5267756,avalderrama@catcolombiasolutions.com,MANUEL ANTONIO GOMEZ RESTREPO
1074,9008252747,AID GROUP SAS,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 94 No. 72A-91 OF 1-503,BOGOTÁ D.C.,BOGOTÁ D.C.,6583762,ivan.cabrera@aidgroup.com.co,IVAN ADOLFO CABRERA CASTAÑEDA
1094,88225309,WILMAN JULIAN RODRIGUEZ SILVA,SERVICIOS E INSTALACIONES ELECTRONICAS SISTEL,INSCRITA EN EL REGISTRO,NaN,P,AV 3 NRO. 14-75 LA PLAYA PISO 1,NORTE DE SANTANDER,CÚCUTA,3153663603,wilsistel@gmail.com,WILMAN JULIAN RODRIGUEZ SILVA
1103,9010124304,SAT STORE S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 80 KM 1.7 PARQUE LA FLORIDA TERRAPUERT...,CUNDINAMARCA,COTA,8764200,administracion@izc.com.co,ANITA SÁNCHEZ MONTEALEGRE
1139,8002272756,CENTRO DE COMUNNICACIONES SAS,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CR. 9 No. 20-13,HUILA,NEIVA,O,NO REGISTRA,LINO JAVIER SIERRA SARMIENTO


In [217]:
Proveedores["ARRENDAMIENTO"] = ""
Proveedores["IMPORTACION"] = ""
Proveedores["FABRICACION"] = ""
Proveedores["COMERCIALIZACION"] = ""
Proveedores["INSTALACION"] = ""

### Proveedores.loc[Proveedores["SECCIONES"].str.contains("COMER"), "COMERCIALIZACION"] =  "COMERCIALIZACION"
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("Ó", "O")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARREDANDAMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARREDAMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDAMINETO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARREDNAMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDMMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDAMIIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENAMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDAMIETO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDEAMIENTO", "ARRENDAMIENTO")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMORTACION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMPORTACFION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMPORTACVION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMPROTACION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMOORTACION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMPOPRTACION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMPORTAVION", "IMPORTACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("FABRICAICON", "FABRICACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCILAIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCIALIOZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMRCIALZIACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMCIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMRCIALIZACIO", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMRCIALIZACDION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCIALZIACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMECIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCIALIZACIO", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCIALIZACDION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCFIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COIMERCIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMPERCIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCIALIZCION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COPMERCIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERACIALIZACION", "COMERCIALIZACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTALACDION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTALAICON", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTLACION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTALÑACION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ISNTALACION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INBSTALACION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTLALACION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTLAACION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTALALCION", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTALCIONA", "INSTALACION")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace(" Y ", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace(" E ", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace(",", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace(" ", "")

Proveedores.loc[Proveedores["SECCIONES"].str.contains("ARRENDAMIENTO", na=False), "ARRENDAMIENTO"] = 'ARRENDAMIENTO'
Proveedores.loc[Proveedores["SECCIONES"].str.contains("IMPORTACION", na=False), "IMPORTACION"] = 'IMPORTACION'
Proveedores.loc[Proveedores["SECCIONES"].str.contains("FABRICACION", na=False), "FABRICACION"] = 'FABRICACION'
Proveedores.loc[Proveedores["SECCIONES"].str.contains("COMERCIALIZACION", na=False), "COMERCIALIZACION"] = 'COMERCIALIZACION'
Proveedores.loc[Proveedores["SECCIONES"].str.contains("INSTALACION", na=False), "INSTALACION"] = 'INSTALACION'

Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("ARRENDAMIENTO", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("IMPORTACION", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("FABRICACION", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("COMERCIALIZACION", "")
Proveedores["SECCIONES"] = Proveedores["SECCIONES"].str.replace("INSTALACION", "")
#Proveedores[Proveedores["SECCIONES"].str.contains("ARRENDAMIENTO", na=False)]
#Proveedores.head(20)
#Proveedores["SECCIONES"].value_counts()

Proveedores.loc[(Proveedores["ARRENDAMIENTO"] == "") &
                (Proveedores["IMPORTACION"] == "") &
                (Proveedores["FABRICACION"] == "") &
                (Proveedores["COMERCIALIZACION"] == "") &
                (Proveedores["INSTALACION"] == "" ), "SECCIONES"] = "NOREGISTRA"
            
Proveedores.loc[~Proveedores["SECCIONES"].isin(["NOREGISTRA", "EXPORTACION"]), "SECCIONES" ] = np.nan

In [218]:
Proveedores



,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,SECCIONES,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION
0,805016312,SEGURIDAD J.J. LTDA - EN LIQUIDACION,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CL. 28N # 6 BN-54,VALLE DEL CAUCA,CALI,O,NO REGISTRA,JOSE JAVIER SEPULVEDA MURILLO,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION
1,9009663510,WIEX TECHNOLOGIES S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CARRERA 57 No. 4B -16,BOGOTÁ D.C.,BOGOTÁ D.C.,3464656,wiextech@gmail.com,WILLIAM HERIBERTO REYES MORENO,,IMPORTACION,,,
2,9008462722,SECURICOL SERVICES S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CARRERA 5 No. 104 -39,BOGOTÁ D.C.,BOGOTÁ D.C.,7446240,mauricio.hernandez@securicol.com,VICTOR MAURICIO HERNÁNDEZ PRIETO,,IMPORTACION,,COMERCIALIZACION,INSTALACION
3,9006584191,OESIA NETWORKS COLOMBIA S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CARRERA 19 B No. 82 -46 PISO 3,BOGOTÁ D.C.,BOGOTÁ D.C.,7560521 3155942028,jcuriber@oesia.com,JULIANA HENAO OVALLE,ARRENDAMIENTO,IMPORTACION,,COMERCIALIZACION,INSTALACION
4,9007087921,WG INGENIERÍA Y COMUNICACIONES E&H S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 13 No. 21 -80,BOGOTÁ D.C.,BOGOTÁ D.C.,3107606149,wihegaca521@gmail.com,WILFRED HERNANDO GARCÍA CARRANZA,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,8301089247,GLOBAL WIDE AREA NETWORK SAS,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CARRERA 19B No. 84 -31 PISO 4,BOGOTÁ D.C,BOGOTÁ D.C.,7425750,info@globalwan.com.co,JEISON EDUARDO CÁRDENAS MONTENEGRO,,,,COMERCIALIZACION,INSTALACION
1203,9007772274,NUEVA ERA TECH S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CARRERA 7ª No. 155ª 80,BOGOTÁ D.C,BOGOTÁ D.C.,3204929654 3212252339,nuevaeratechsas@gmail.com,MILTON RIAÑO HERNANDEZ,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION
1204,8001576299,MOUSE STAR S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,"PARQUE INDUSTRIAL PORTOS SABANA 80 BG 52 KM 2,...",CUNDINAMARCA,COTA,8764717 8764526,mariae@mousestar.com.co,MARIA ESTHER HERRERA AVILA,,IMPORTACION,,COMERCIALIZACION,
1205,8060148698,SERVICIOS INTEGRADOS SI LTDA,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,"Manzanillo, kilómetro 10, Centro Comercial las...",BOLIVAR,CARTAGENA,3163349228,info@serviciosintegrados.com.co,GABRIEL FRANCISCO VERGARA LAGO,ARRENDAMIENTO,IMPORTACION,,COMERCIALIZACION,INSTALACION


## Consolidar una sola lista

In [247]:
Proveedores.rename(columns= {
    'NIT': 'NIT O CC',
    'TIPO OFICINA': 'TIPO DE OFICINA',
    'CORREO ELECTRONICO - RUES': 'CORREO ELECTRÓNICO',
    'REPRESENTANTE LEGAL O ADMINISTRADOR': 'REPRESENTANTE LEGAL'}, inplace=True)

In [248]:
[col for col in Servicios.columns if col not in Proveedores.columns]

['TIPO DE TRÁMITE']

In [249]:
[col for col in Proveedores.columns if col not in Servicios.columns]

['SECCIONES',
 'ARRENDAMIENTO',
 'IMPORTACION',
 'FABRICACION',
 'COMERCIALIZACION',
 'INSTALACION']

In [379]:
empresas_VSP_df = pd.concat([Servicios, Proveedores], axis=0)
empresas_VSP_df.reset_index(inplace=True, drop=True)
empresas_VSP_df.info()
#empresas_VSP_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2565 entries, 0 to 2564
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NIT O CC             2565 non-null   object
 1   RAZÓN SOCIAL         2565 non-null   object
 2   ACRÓNIMO             2560 non-null   object
 3   TIPO DE SERVICIO     2565 non-null   object
 4   TIPO DE TRÁMITE      1357 non-null   object
 5   TIPO DE OFICINA      2565 non-null   object
 6   DIRECCIÓN            2565 non-null   object
 7   DEPARTAMENTO         2565 non-null   object
 8   CIUDAD               2565 non-null   object
 9   TELÉFONO             2565 non-null   object
 10  CORREO ELECTRÓNICO   2558 non-null   object
 11  REPRESENTANTE LEGAL  2564 non-null   object
 12  SECCIONES            13 non-null     object
 13  ARRENDAMIENTO        1207 non-null   object
 14  IMPORTACION          1207 non-null   object
 15  FABRICACION          1207 non-null   object
 16  COMERC

- Departamentos
- Municipios / Ciudad / Localidad

In [380]:
# Arreglar el nombre de los departamentos
empresas_VSP_df['DEPARTAMENTO'].replace({
        'VILALVICENCIO': 'VILLAVICENCIO',
        'BOGOTA': 'BOGOTÁ D.C.',
        'BOGOTÁ D.C': 'BOGOTÁ D.C.',
        'SANTANDER DEL SUR': 'SANTANDER',
        'VALLE': 'VALLE DEL CAUCA',
        'ATLANTICO':'ATLÁNTICO',
        'BOLIVAR': 'BOLÍVAR',
        'BOYACA': 'BOYACÁ',
        'QUINDIO': 'QUINDÍO',
        'SAN ANDRES ISLAS': 'SAN ANDRES',
        }, inplace=True)

In [381]:
# Revisar cuales departamentos están mal
empresas_VSP_df['DEPARTAMENTO'].value_counts().sort_index(ascending=True)


AMAZONAS                 1
ANTIOQUIA              253
ARAUCA                   1
ATLÁNTICO              131
BOGOTÁ D.C.           1471
BOLÍVAR                 28
BOYACÁ                  20
CALDAS                  12
CAQUETÁ                  3
CASANARE                 9
CAUCA                    4
CESAR                   19
CHÍA                     1
CORDOBA                  1
COTA                     1
CUNDINAMARCA            84
CÓRDOBA                  5
DOSQUEBRADAS             1
GUAJIRA                  7
GUAVIARE                 1
HUILA                   23
MAGDALENA               23
MEDELLIN                 2
META                    26
NARIÑO                   9
NORTE DE SANTANDER      26
PIEDECUESTA              1
PUTUMAYO                 1
QUINDÍO                 16
RISARALDA               41
SAN ANDRES               2
SANTA MARTA              1
SANTANDER               87
SOACHA                   1
SUCRE                    2
TOLIMA                  34
VALLE DEL CAUCA        215
V

In [382]:
# Ajustar los  Departamentos y ciudades que están invertidos:
empresas_VSP_df2 = empresas_VSP_df.loc[empresas_VSP_df['DEPARTAMENTO'].isin(['DOSQUEBRADAS', 'CHÍA', 'PIEDECUESTA', 'VILLAVICENCIO', 
                                                                             'SOACHA', 'SANTA MARTA', 'COTA', 'MEDELLIN', 'BOGOTA', ])]

Serie = empresas_VSP_df2['CIUDAD']
empresas_VSP_df2['CIUDAD'] = empresas_VSP_df2['DEPARTAMENTO']
empresas_VSP_df2['DEPARTAMENTO'] = Serie
empresas_VSP_df.loc[empresas_VSP_df2.index] = empresas_VSP_df2

#empresas_VSP_df.loc[empresas_VSP_df['DEPARTAMENTO'].isin(['DOSQUEBRADAS', 'CHÍA', 'PIEDECUESTA', 'VILLAVICENCIO', 'SOACHA',
                                                          #'SANTA MARTA', 'COTA', 'MEDELLIN', 'BOGOTA']), ["CIUDAD", "DEPARTAMENTO"]]
#empresas_VSP_df['DEPARTAMENTO'].value_counts()

C:\Users\gatew\AppData\Local\Temp\ipykernel_11416\618916625.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_VSP_df2['CIUDAD'] = empresas_VSP_df2['DEPARTAMENTO']
C:\Users\gatew\AppData\Local\Temp\ipykernel_11416\618916625.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_VSP_df2['DEPARTAMENTO'] = Serie


In [383]:
cities = empresas_VSP_df['CIUDAD'].value_counts().sort_index(ascending=True).index.to_list()
print(len(cities))
cities[0:20]

115


['ACOPI YUMBO',
 'AGUACHICA',
 'ARMENIA',
 'BARANQUILLA',
 'BARRANCABERMEJA',
 'BARRANQUILLA',
 'BARRANQUILLA ATLANTICO',
 'BELLO',
 'BOGOTA',
 'BOGOTÁ D.C',
 'BOGOTÁ D.C.',
 'BOSCONIA',
 'BUCARAMANGA',
 'BUENAVENTURA',
 'BUGA',
 'CAJICA',
 'CAJICÁ',
 'CALI',
 'CANDELARIA',
 'CARTAGENA']

In [384]:
# Arreglar el nombre de las ciudades
empresas_VSP_df['CIUDAD'].replace({
        ' -':'',
        ' ATLANTICO':'',
        'BARANQUILLA': 'BARRANQUILLA',
        'BOGOTA': 'BOGOTÁ D.C.',
        'BOGOTÁ D.C': 'BOGOTÁ D.C.',
        '´BOGOTÁ D.C': 'BOGOTÁ D.C.',
        'CAJICA': 'CAJICÁ',
        'CHIA': 'CHÍA',
        'CUCUTA': 'CÚCUTA',
        'DOS QUEBRADAS': 'DOSQUEBRADAS',
        'DORADA':'LA DORADA',
        'FACATATIVA':'FACATATIVÁ',
        'GIRON':'GIRÓN',
        'IBAGUE':'IBAGUÉ',
        'MANIZALEZ':'MANIZALES',
        'MEDELLIN': 'MEDELLÍN',
        'MONTERIA':  'MONTERÍA',
        'POPAYAN': 'POPAYÁN',
        'PUERTO BOYACA':'PUERTO BOYACÁ',
        'SAN ANDRES ISLAS': 'SAN ANDRES',
        'SOHACHA':'SOACHA',
        'TOCANCIPA': 'TOCANCIPÁ',
        'ZIPAQUIRA': 'ZIPAQUIRÁ'}, inplace=True)

empresas_VSP_df['CIUDAD'].value_counts().shape

(95,)

### Están registrados en el RUES?

#### Ajustar el formato de NITs de las empresas registradas en RENOVA

In [385]:
empresas_VSP_df['NIT O CC'].str.len().describe()

count    2564.000000
mean        9.930187
std         0.385305
min         7.000000
25%        10.000000
50%        10.000000
75%        10.000000
max        11.000000
Name: NIT O CC, dtype: float64

In [386]:
empresas_VSP_df['NIT O CC']

0       8600654643
1       9010567354
2       8603518121
3       9012112784
4       9012498921
           ...    
2560    8301089247
2561    9007772274
2562    8001576299
2563    8060148698
2564    9012932366
Name: NIT O CC, Length: 2565, dtype: object

In [387]:
# Verificar que el último dígito es un dígito de verificacion:
calcular_digito_ver(803040103)

4

In [388]:
#Seleccionar solamente los 9 dígitos que nos interesan, indexando del ultimo al primero, porque hay secuencias más largas.
empresas_VSP_df['NIT O CC'].str[-10:-1]

0       860065464
1       901056735
2       860351812
3       901211278
4       901249892
          ...    
2560    830108924
2561    900777227
2562    800157629
2563    806014869
2564    901293236
Name: NIT O CC, Length: 2565, dtype: object

In [389]:
#Seleccionar solamente las empresas que tienen un NIT de más de 9 caracteres
empresas2 = empresas_VSP_df.loc[empresas_VSP_df['NIT O CC'].str.len()>9].copy()
empresas2.shape

(2469, 18)

In [390]:
# Calcular el dígito de verificacion para compararlo con el último dígito existente
empresas2['digito_ver'] = empresas2['NIT O CC'].str[-1].str.strip()
empresas2['NIT O CC'] = empresas2['NIT O CC'].str[-10:-1]
empresas2['digito_ver2'] = empresas2['NIT O CC'].apply(calcular_digito_ver)
#empresas2['digito_ver'] = empresas2['NIT O CC'].str[-10:-1].apply(calcular_digito_ver)
empresas2[['NIT O CC', 'digito_ver', 'digito_ver2']]

,NIT O CC,digito_ver,digito_ver2
0,860065464,3,3
1,901056735,4,4
2,860351812,1,1
3,901211278,4,4
4,901249892,1,1
...,...,...,...
2560,830108924,7,7
2561,900777227,4,4
2562,800157629,9,9
2563,806014869,8,8


In [391]:
#Cuantos NITS no coincidieron con el dígito de verificacion?
empresas2[(empresas2['digito_ver'].astype(int) - empresas2['digito_ver2'].astype(int)) !=0].shape

(27, 20)

In [392]:
# Lista de filas que tienen estos NITS
indice = empresas2[(empresas2['digito_ver'].astype(int) - empresas2['digito_ver2'].astype(int)) ==0].index
indice

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2554, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564],
           dtype='int64', length=2442)

In [393]:
# Imputar NITs a partir del cálculo anterior
empresas_VSP_df.loc[indice, 'NIT O CC'] = empresas2.loc[indice, 'NIT O CC']

In [394]:
# Seleccionar NITS que están en el registro pero cuyo digito de verificación está mal
#empresas2.drop(indice, inplace=True)
empresas2 = empresas_VSP_df.loc[(empresas_VSP_df['NIT O CC'].str.len()==10) & (empresas_VSP_df['NIT O CC'].str[0].isin(['8', '9']))]
empresas2.shape

(13, 18)

In [395]:
# Imputar otros Nits que se han encontrado
empresas2['NIT O CC'] = empresas2['NIT O CC'].str[-10:-1]
empresas_VSP_df.loc[empresas2.index, 'NIT O CC'] = empresas2['NIT O CC']

C:\Users\gatew\AppData\Local\Temp\ipykernel_11416\4048342920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas2['NIT O CC'] = empresas2['NIT O CC'].str[-10:-1]


In [396]:
# Arreglar el numero de dígitos
empresas_VSP_df['NIT O CC'].str.len().describe()

count    2564.000000
mean        8.972699
std         0.235454
min         7.000000
25%         9.000000
50%         9.000000
75%         9.000000
max        11.000000
Name: NIT O CC, dtype: float64

In [397]:
# Incrementar el número de dígitos en el string para poderlos comparar con los del RUES
empresas_VSP_df['NIT O CC'] = empresas_VSP_df['NIT O CC'].apply(extender_nit_string)
empresas_VSP_df['NIT O CC'].str.len().describe()

count    2565.0
mean       14.0
std         0.0
min        14.0
25%        14.0
50%        14.0
75%        14.0
max        14.0
Name: NIT O CC, dtype: float64

#### Match de NITS. Se hace un Left Join (Merge) de ambos Dataframes

In [398]:
Registro_RUES[Registro_RUES["numero_identificacion"].str.contains('901293236')]

,clase_identificacion,numero_identificacion,nit_propietario,razon_social,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,ciiu3,ciiu4,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,fecha_cancelacion,categoria_matricula


In [416]:
empresas_VSP_df2 = empresas_VSP_df.merge(Registro_RUES,  how='left', left_on='NIT O CC', right_on='nit_propietario')
print(empresas_VSP_df2.shape)
empresas_VSP_df2.tail()


(18860, 33)


,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,SECCIONES,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,clase_identificacion,numero_identificacion,nit_propietario,razon_social,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,ciiu3,ciiu4,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,fecha_cancelacion,categoria_matricula
18855,00000800157629,MOUSE STAR S.A.S.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,"PARQUE INDUSTRIAL PORTOS SABANA 80 BG 52 KM 2,...",CUNDINAMARCA,COTA,8764717 8764526,mariae@mousestar.com.co,MARIA ESTHER HERRERA AVILA,NaN,,IMPORTACION,,COMERCIALIZACION,,NIT,800157629,00000800157629,MOUSE STAR SAS,4651,NaN,NaN,NaN,BOGOTA,491451,2022,2022-03-28,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
18856,00000806014869,SERVICIOS INTEGRADOS SI LTDA,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,"Manzanillo, kilómetro 10, Centro Comercial las...",BOLÍVAR,CARTAGENA,3163349228,info@serviciosintegrados.com.co,GABRIEL FRANCISCO VERGARA LAGO,NaN,ARRENDAMIENTO,IMPORTACION,,COMERCIALIZACION,INSTALACION,NIT,806014869,00000806014869,SERVICIOS INTEGRADOS S.I. LTDA.,8129,8020,NaN,NaN,BARRANQUILLA,711875,2022,2022-03-29,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
18857,00000806014869,SERVICIOS INTEGRADOS SI LTDA,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,"Manzanillo, kilómetro 10, Centro Comercial las...",BOLÍVAR,CARTAGENA,3163349228,info@serviciosintegrados.com.co,GABRIEL FRANCISCO VERGARA LAGO,NaN,ARRENDAMIENTO,IMPORTACION,,COMERCIALIZACION,INSTALACION,NIT,806014869,00000806014869,SERVICIOS INTEGRADOS SI LTDA,8129,8020,NaN,NaN,CARTAGENA,41389002,2022,2022-03-29,ACTIVA,NaN,ESTABLECIMIENTO DE COMERCIO
18858,00000806014869,SERVICIOS INTEGRADOS SI LTDA,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,"Manzanillo, kilómetro 10, Centro Comercial las...",BOLÍVAR,CARTAGENA,3163349228,info@serviciosintegrados.com.co,GABRIEL FRANCISCO VERGARA LAGO,NaN,ARRENDAMIENTO,IMPORTACION,,COMERCIALIZACION,INSTALACION,NIT,806014869,00000806014869,SERVICIOS INTEGRADOS SI LTDA.,5512,NaN,NaN,NaN,CARTAGENA,18590902,2008,2008-01-11,CANCELADA,20150713.0,ESTABLECIMIENTO DE COMERCIO
18859,00000901293236,G&G TECHLIFE S.A.S,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,Carrera 59 No. 160 - 86,CUNDINAMARCA,BOGOTÁ D.C.,7732310 3006571104,feloseba@icloud.com,LUIS FELIPE GONZALEZ PARDO,NaN,,IMPORTACION,,COMERCIALIZACION,INSTALACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [417]:
empresas_VSP_df2['Tiene_Registro_Mercantil_RUES'] = ~empresas_VSP_df2['numero_identificacion'].isna()
empresas_VSP_df2['Tiene_Registro_Mercantil_RUES'].value_counts()

True     17884
False      976
Name: Tiene_Registro_Mercantil_RUES, dtype: int64

In [418]:
#empresas_VSP_df2 = empresas_VSP_df2[empresas_VSP_df2['estado_matricula']=='ACTIVA']
empresas_VSP_df2.drop(columns = {'numero_identificacion','nit_propietario','razon_social','ciiu3', 'ciiu4', 'fecha_cancelacion'}, inplace=True)

In [ ]:
empresas_VSP_df2 = empresas_VSP_df2.sort_values(by=['ultimo_ano_renovado', 'fecha_renovacion'], ascending=False).sort_values(by=['estado_matricula'])

In [419]:
empresas_VSP_df2.drop_duplicates(inplace=True)
print(empresas_VSP_df2.shape)

(18523, 28)


In [428]:
empresas_VSP_df2.drop_duplicates(subset=['NIT O CC', 'RAZÓN SOCIAL', 'TIPO DE SERVICIO',
       'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'camara_comercio'], keep='first', inplace=True)
print(empresas_VSP_df2.shape)

(7666, 28)


In [429]:
empresas_VSP_df2['Tiene_Registro_Mercantil_RUES'].value_counts()

True     6690
False     976
Name: Tiene_Registro_Mercantil_RUES, dtype: int64

In [430]:
empresas_VSP_df2['estado_matricula'].value_counts()

ACTIVA                                     5318
CANCELADA                                  1348
MATRÍCULA CANCELADA LEY 1429                 22
INACTIVA POR CIERRE DEL ESTABLECIMIENTO       2
Name: estado_matricula, dtype: int64

In [431]:
empresas_VSP_df2.columns

Index(['NIT O CC', 'RAZÓN SOCIAL', 'ACRÓNIMO', 'TIPO DE SERVICIO',
       'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'DEPARTAMENTO',
       'CIUDAD', 'TELÉFONO', 'CORREO ELECTRÓNICO', 'REPRESENTANTE LEGAL',
       'SECCIONES', 'ARRENDAMIENTO', 'IMPORTACION', 'FABRICACION',
       'COMERCIALIZACION', 'INSTALACION', 'clase_identificacion',
       'cod_ciiu_act_econ_pri', 'cod_ciiu_act_econ_sec', 'camara_comercio',
       'matricula', 'ultimo_ano_renovado', 'fecha_renovacion',
       'estado_matricula', 'categoria_matricula',
       'Tiene_Registro_Mercantil_RUES'],
      dtype='object')

In [16]:
# Ver solamente las empresas registradas en RENOVAL
empresas_VSP_df2.drop_duplicates(subset=['NIT O CC','RAZÓN SOCIAL', 'TIPO DE SERVICIO', 'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'CIUDAD', 'CORREO ELECTRÓNICO', 'REPRESENTANTE LEGAL']).shape#['estado_matricula'].value_counts()

(2563, 29)

In [25]:
# Cuantas de las empresas en RENOVA tienen registro activo o cancelado? (la diferencia con el conteo anterior son las que no estan registradas)
empresas_VSP_df2.drop_duplicates(subset=['NIT O CC','RAZÓN SOCIAL', 'TIPO DE SERVICIO', 'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'CIUDAD', 'CORREO ELECTRÓNICO', 'REPRESENTANTE LEGAL'])['estado_matricula'].value_counts()

ACTIVA                          1340
CANCELADA                        241
MATRÍCULA CANCELADA LEY 1429       6
Name: estado_matricula, dtype: int64

In [24]:
# Cuantas empresas estan activas y no están repetidas (tienen diferentes sedes)?
empresas_VSP_df2[(empresas_VSP_df2['estado_matricula'] == 'ACTIVA') & 
                 (empresas_VSP_df2['ultimo_ano_renovado'].isin(['2021', '2022']))].drop_duplicates(subset=['NIT O CC','RAZÓN SOCIAL', 'TIPO DE SERVICIO', 'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'CIUDAD', 'CORREO ELECTRÓNICO', 'REPRESENTANTE LEGAL']).shape

(1187, 29)

In [31]:
empresas_VSP_df2 = empresas_VSP_df2.sort_values(by=['ultimo_ano_renovado', 'fecha_renovacion'], ascending=False).sort_values(by=['NIT O CC', 'estado_matricula'])
empresas_VSP_df2.head(3)

,Unnamed: 0,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,SECCIONES,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,clase_identificacion,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,categoria_matricula,Tiene_Registro_Mercantil_RUES
5858,15339,00000001405491,LEONARDO SANCHEZ GIRALDO,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 25 No. 99-33,BOGOTÁ D.C.,BOGOTÁ D.C.,4215688,leosangi@gmail.com,LEONARDO SANCHEZ GIRALDO,NaN,NaN,IMPORTACION,NaN,COMERCIALIZACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4717,12178,00000002937583,RAFAEL GONZALEZ DAZA,NO REGISTRA,DEPARTAMENTO DE SEGURIDAD,RENOVACIÓN,P,CARRERA 9 N° 13B-127 OFICINA 305,CESAR,VALLEDUPAR,5706363,ragodaza@hotmail.com,RAFAEL GONZALEZ DAZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5777,15246,00000004908161,HOME DAGOBERTO,HOME SECURITY,INSCRITA EN EL REGISTRO,NaN,P,CR. 10 No. 15-78/68 OF. 419,BOGOTÁ D.C.,BOGOTÁ D.C.,3153501366 3138509717,homeseguridadelectronica@hotmail.com,DAGO BERTO HOME,NaN,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


### Guardar Datos

In [1]:
pwd

'd:\\DS4A\\SuperVigilancia\\OneDrive - Superintendencia de Vigilancia\\Proyecto Observatorio 2022\\5.Notebooks\\1_Servicios_Vigilados'

In [32]:
ls

 Volume in drive D has no label.
 Volume Serial Number is 5E36-13E0

 Directory of d:\DS4A\SuperVigilancia\OneDrive - Superintendencia de Vigilancia\Proyecto Observatorio 2022\4.Bases_de_datos_Externas\Camaras_de_Comercio-Registro_Unico_de_Empresas_RUES

10/20/2022  02:35 PM    <DIR>          .
10/20/2022  02:35 PM    <DIR>          ..
10/14/2022  06:04 PM    <DIR>          0.Raw_dataset
10/14/2022  06:07 PM    <DIR>          1.Clean_dataset
10/20/2022  07:39 PM    <DIR>          2.Processed_dataset
10/13/2022  04:22 PM           121,869 actividades_codigo_ciiu_2021.xlsx
10/13/2022  04:30 PM            95,972 CIIU.json
10/13/2022  04:14 PM            56,815 CIIU.txt
10/14/2022  04:14 PM            34,214 CIIU_Clase.csv
10/20/2022  02:35 PM            55,132 CIIU_Clase2.csv
09/20/2022  07:43 PM            70,594 Tabla correlativa CIIU 4 A.C. (2020) - CIIU 4 A.C. (2021).xlsx
10/13/2022  05:27 PM           113,758 Tabla correlativa CIIU.xlsx
09/21/2022  10:51 AM           226,127 TC_CIIU4

In [34]:
# Save the recovered data from the SQL request to the Datos Abiertos API: 
# (Uncomment to save, comment after save in order to not damage the previous file)
# Este archivo queda en 4.Bases_de_datos_Externas\Camaras_de_Comercio-Registro_Unico_de_Empresas_RUES\2.Processed_dataset
empresas_VSP_df2.to_csv('./2.Processed_dataset/Vigilados_registrados_RUES.csv')

In [11]:
empresas_VSP_df3 = pd.read_csv("./2.Processed_dataset/Vigilados_registrados_RUES.csv", encoding = 'utf-8', dtype='str', sep=',')
print(empresas_VSP_df3.shape)
empresas_VSP_df3.head()

(7666, 30)


,Unnamed: 0.1,Unnamed: 0,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,SECCIONES,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,clase_identificacion,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,categoria_matricula,Tiene_Registro_Mercantil_RUES
0,5858,15339,00000001405491,LEONARDO SANCHEZ GIRALDO,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 25 No. 99-33,BOGOTÁ D.C.,BOGOTÁ D.C.,4215688,leosangi@gmail.com,LEONARDO SANCHEZ GIRALDO,NaN,NaN,IMPORTACION,NaN,COMERCIALIZACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,4717,12178,00000002937583,RAFAEL GONZALEZ DAZA,NO REGISTRA,DEPARTAMENTO DE SEGURIDAD,RENOVACIÓN,P,CARRERA 9 N° 13B-127 OFICINA 305,CESAR,VALLEDUPAR,5706363,ragodaza@hotmail.com,RAFAEL GONZALEZ DAZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,5777,15246,00000004908161,HOME DAGOBERTO,HOME SECURITY,INSCRITA EN EL REGISTRO,NaN,P,CR. 10 No. 15-78/68 OF. 419,BOGOTÁ D.C.,BOGOTÁ D.C.,3153501366 3138509717,homeseguridadelectronica@hotmail.com,DAGO BERTO HOME,NaN,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,5821,15296,00000005538070,CIRO ALFONSO PABON JEREZ,ADEMCO TELEGUARD,INSCRITA EN EL REGISTRO,NaN,P,AV. LOS BUCAROS # 2-80,SANTANDER,BUCARAMANGA,6418021,teleguard@intercable.net.co,CIRO ALFONSO PABON JEREZ,NaN,NaN,IMPORTACION,NaN,COMERCIALIZACION,INSTALACION,NIT,9999,NaN,BUCARAMANGA,410077977,1979,1979-03-14,CANCELADA,ESTABLECIMIENTO DE COMERCIO,True
4,5844,15323,00000006187299,JESUS ANTONIO VILLAMIL ARENAS,ALARMAS SENTRY,INSCRITA EN EL REGISTRO,NaN,P,CR. 31 # 23-13,VALLE DEL CAUCA,PALMIRA,11111,NO REGISTRA,JESUS ANTONIO VILLAMIL ARENAS,NaN,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,CEDULA DE CIUDADANIA,5169,NaN,PALMIRA,18846,1996,1996-02-07,CANCELADA,ESTABLECIMIENTO DE COMERCIO,True


### Cual es la clasificación de Actividad

In [15]:
PATH = 'CIIU_CLase2.csv'
#CIIU_codes_df = pd.read_csv(PATH, encoding = 'utf-8', dtype='str', sep=',' )
CIIU_codes_df = pd.read_csv(PATH, encoding = 'latin3', dtype='str', sep=',' )
print(CIIU_codes_df.shape)
CIIU_codes_df.head(3)

(844, 4)


,Clase,Descripción,Unnamed: 2,Unnamed: 3
0,100,"Agricultura, ganadería, caza y actividades de ...",NaN,NaN
1,110,Cultivos agrícolas transitorios,NaN,NaN
2,111,"Cultivo de cereales (excepto arroz), legumbres...",NaN,NaN


In [17]:
# Eliminar columnas innecesarias
CIIU_codes_df.drop(columns=['Unnamed: 2', 'Unnamed: 3'], inplace=True)
# Arreglar el formato a 4 d'igitos
CIIU_codes_df['Clase'] = CIIU_codes_df['Clase'].apply(limpiar_ciuu_code)
# Eliminar repetidos (division, grupo y clase que tienen el mismo c'odigo)
CIIU_codes_df.drop_duplicates(subset="Clase", inplace=True)
print(CIIU_codes_df.shape)
CIIU_codes_df.head()

(688, 2)


,Clase,Descripción
0,0100,"Agricultura, ganadería, caza y actividades de ..."
1,0110,Cultivos agrícolas transitorios
2,0111,"Cultivo de cereales (excepto arroz), legumbres..."
3,0112,Cultivo de arroz
4,0113,"Cultivo de hortalizas, raíces y tubérculos"


In [18]:
#Crear un diccionario para hacer la asignación de descripciones de las actividades
CIIU_clase_codes_dict = CIIU_codes_df.set_index('Clase', drop=True).to_dict()
CIIU_clase_codes_dict = CIIU_clase_codes_dict['Descripción']
#CIIU_clase_codes_dict

In [19]:
# Para cada empresa registrada, seleccionar solamente una sede. (La ultima registada)
empresas_VSP_df3 = empresas_VSP_df3.drop_duplicates(subset=['NIT O CC','RAZÓN SOCIAL', 'TIPO DE SERVICIO', 'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'CIUDAD', 'CORREO ELECTRÓNICO', 'REPRESENTANTE LEGAL'])
empresas_VSP_df3.shape

(2563, 30)

In [20]:
#Verificar que los datos sean la ultima sede registrada. 
empresas_VSP_df3[empresas_VSP_df3['NIT O CC']=='00000013886938']
#empresas_VSP_df3[empresas_VSP_df3['NIT O CC']=='00000013886938'].sort_values(by=['ultimo_ano_renovado', 'fecha_renovacion'], ascending=False).sort_values(by=['NIT O CC', 'estado_matricula'])

,Unnamed: 0.1,Unnamed: 0,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,SECCIONES,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,clase_identificacion,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,categoria_matricula,Tiene_Registro_Mercantil_RUES
17,5873,15357,00000013886938,MIGUEL ANGEL TORO JARAMILLO,COMERCIALIZADORA CIBERDRIVERS,INSCRITA EN EL REGISTRO,NaN,P,CALLE 37 No. 16 -04 LOCAL 306,SANTANDER,BUCARAMANGA,3216478183,contabilidad@systemspy.net.co,MIGUEL ANGEL TORO JARAMILLO,NaN,NaN,IMPORTACION,NaN,COMERCIALIZACION,INSTALACION,CEDULA DE CIUDADANIA,4651,NaN,BUCARAMANGA,410297845,2013,2013-05-09,ACTIVA,ESTABLECIMIENTO DE COMERCIO,True


In [27]:
empresas_VSP_df3['cod_ciiu_desc'] = empresas_VSP_df3['cod_ciiu_act_econ_pri'].map(CIIU_clase_codes_dict)
empresas_VSP_df3[['cod_ciiu_act_econ_pri', 'cod_ciiu_desc']].value_counts().tail().to_frame()

,,0
cod_ciiu_act_econ_pri,cod_ciiu_desc,
4729,"Comercio al por menor de otros productos alimenticios n.c.p., en establecimientos especializados",1
4322,"Instalaciones de fontanería, calefacción y aire acondicionado",1
4220,Construcción de proyectos de servicio público,1
3821,Tratamiento y disposición de desechos no peligrosos,1
4751,Comercio al por menor de productos textiles en establecimientos especializados,1


In [ ]:
Aqui voy. 

In [36]:
CIIU_codes_df.loc[CIIU_codes_df['Clase'].str[:3] == '011', 'Descripción'].values[0]

'Cultivos agrícolas transitorios'

In [ ]:
empresas_VSP_df3.loc[empresas_VSP_df3['cod_ciiu_desc'].isna(), 'cod_ciiu_desc'] = CIIU_codes_df.loc[CIIU_codes_df['Clase'].str[:3] == empresas_VSP_df3.loc[empresas_VSP_df3['cod_ciiu_desc'].isna(), 'cod_ciiu_act_econ_pri'], 'Descripción'].values[0]

In [37]:
CIIU_codes_df.loc[CIIU_codes_df['Clase'].str[:3] == empresas_VSP_df3.loc[empresas_VSP_df3['cod_ciiu_desc'].isna(), 'cod_ciiu_act_econ_pri'], 'Descripción'].values[0]

ValueError: Can only compare identically-labeled Series objects

In [31]:
empresas_VSP_df3['cod_ciiu_act_econ_pri'].str[:3].value_counts()

801    483
465    157
855     56
802     56
492     51
      ... 
106      1
651      1
201      1
559      1
811      1
Name: cod_ciiu_act_econ_pri, Length: 145, dtype: int64

In [28]:
empresas_VSP_df3[empresas_VSP_df3['NIT O CC']=='00000001405491']

,Unnamed: 0.1,Unnamed: 0,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,SECCIONES,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,clase_identificacion,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,categoria_matricula,Tiene_Registro_Mercantil_RUES,cod_ciiu_desc
0,5858,15339,00000001405491,LEONARDO SANCHEZ GIRALDO,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 25 No. 99-33,BOGOTÁ D.C.,BOGOTÁ D.C.,4215688,leosangi@gmail.com,LEONARDO SANCHEZ GIRALDO,NaN,NaN,IMPORTACION,NaN,COMERCIALIZACION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN


In [102]:
empresas_VSP_df3[empresas_VSP_df3['cod_ciiu_act_econ_pri']=='5169']

,Unnamed: 0,NIT O CC,RAZÓN SOCIAL,ACRÓNIMO,TIPO DE SERVICIO,TIPO DE TRÁMITE,TIPO DE OFICINA,DIRECCIÓN,DEPARTAMENTO,CIUDAD,TELÉFONO,CORREO ELECTRÓNICO,REPRESENTANTE LEGAL,SECCIONES,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,clase_identificacion,cod_ciiu_act_econ_pri,cod_ciiu_act_econ_sec,camara_comercio,matricula,ultimo_ano_renovado,fecha_renovacion,estado_matricula,categoria_matricula,Tiene_Registro_Mercantil_RUES
1061,1241,00000800233036,METRO ALARMAS LTDA,NO REGISTRA,EMPRESA DE VIGILANCIA Y SEGURIDAD PRIVADA,RENOVACIÓN,P,CARRERA 78 No. 45A -94,ANTIOQUIA,MEDELLÍN,4481000,dircontabilidad@metroalarmas.com,ALEJANDRO ARCILA RESTREPO,NaN,NaN,NaN,NaN,NaN,NaN,NIT,5169,NaN,CALI,560052,2002,2002-03-22,CANCELADA,AGENCIA,True
5844,15323,00000006187299,JESUS ANTONIO VILLAMIL ARENAS,ALARMAS SENTRY,INSCRITA EN EL REGISTRO,NaN,P,CR. 31 # 23-13,VALLE DEL CAUCA,PALMIRA,11111,NO REGISTRA,JESUS ANTONIO VILLAMIL ARENAS,NaN,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,CEDULA DE CIUDADANIA,5169,NaN,PALMIRA,18846,1996,1996-02-07,CANCELADA,ESTABLECIMIENTO DE COMERCIO,True
6025,15634,00000800232762,TELE CARS S A,TELECARS SA,INSCRITA EN EL REGISTRO,NaN,P,AV. BOYACA # 3-24,BOGOTÁ D.C.,BOGOTÁ D.C.,4191612 5655555,gerencia@telecars.net,HERNANDO ALBERTO CASTANEDA OSORIO,NaN,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,NIT,5169,NaN,MEDELLIN,29621502,1999,1999-03-29,MATRÍCULA CANCELADA LEY 1429,AGENCIA,True
6175,15877,00000811021448,MACRO ALARMAS LTDA EN LIQUIDACION,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,CALLE 51 No. 54-42,ANTIOQUIA,MEDELLÍN,3101100,macroalarmas@une.net.co,CARLOS ALBERTO GOMEZ MONTOYA,NaN,ARRENDAMIENTO,IMPORTACION,FABRICACION,COMERCIALIZACION,INSTALACION,NIT,5169,NaN,MEDELLIN,32222302,2011,2011-03-31,ACTIVA,ESTABLECIMIENTO DE COMERCIO,True
6673,17039,00000890311875,SCHNEIDER ELECTRIC DE COLOMBIA S.A.,NO REGISTRA,INSCRITA EN EL REGISTRO,NaN,P,COMPLEJO INDUSTRIAL CELTA TRADE PARK BODEGA 10...,CUNDINAMARCA,FUNZA,4198484,paola.valero@schneider-electric.com,MARCOS ALVARENGA MATIAS,NaN,NaN,IMPORTACION,NaN,COMERCIALIZACION,NaN,NIT,5169,NaN,MEDELLIN,10343302,2011,2011-03-31,CANCELADA,AGENCIA,True


Asignar a los Ciuu que no se encontraron las descripciones de nivel superior (division, los dos primeros digitos): 

In [30]:
empresas_VSP_df3[empresas_VSP_df3['cod_ciiu_desc'].isna()]['cod_ciiu_act_econ_pri'].value_counts()

5169    3
6426    3
7499    3
5249    2
5165    2
6599    2
5245    2
5243    2
5133    1
3190    1
7421    1
5241    1
6339    1
8060    1
5190    1
7290    1
6425    1
5119    1
5131    1
5239    1
Name: cod_ciiu_act_econ_pri, dtype: int64

### Clasificación por sector económico

## Step 3. Data Augmentation

### Cuántos contratos tiene en RENOVA?

In [ ]:
### Cuántos contratos tiene en RENOVA

In [ ]:
# Ver solamente las empresas registradas en RENOVAL
empresas_RUES_df3 = empresas_RUES_df2.drop_duplicates(subset=['NIT O CC','RAZÓN SOCIAL', 'TIPO DE SERVICIO', 'TIPO DE TRÁMITE', 'TIPO DE OFICINA', 'DIRECCIÓN', 'CIUDAD', 'CORREO ELECTRÓNICO', 'REPRESENTANTE LEGAL']).shape#['estado_matricula'].value_counts()

### Tienen Contratos en SECOP?

### Guardar Archivos

In [ ]:
# Se guarda la version de trabajo del archivo de Servicios Vigilados agui:
## 3.Bases_de_datos_Internas\RENOVA-Datos_Abiertos-Empresas_de_Vigilancia\2.Processed_dataset
empresas_VSP_df2.to_csv('./2.Processed_dataset/Vigilados_registrados_RUES.csv')

# Step 4 Geocoding

# Step 5 Geo-Clustering usando polígonos